#### Import necessary libraries

In [1]:
from numpy import zeros, ones, expand_dims, asarray 

from numpy.random import randn, randint 

from keras.datasets import fashion_mnist

from keras.optimizers import Adam

from keras.models import Model, load_model 

from keras.layers import Input, Dense, Reshape, Flatten 

from keras.layers import Conv2D, Conv2DTranspose, Concatenate 

from keras.layers import LeakyReLU, Dropout, Embedding 

from keras.layers import BatchNormalization, Activation 

from keras import initializers 

from keras.initializers import RandomNormal 

from keras.optimizers import Adam, RMSprop, SGD 

from matplotlib import pyplot 

import numpy as np 

from math import sqrt 

In [2]:
(X_train, _), (_, _) = fashion_mnist.load_data() 

X_train = X_train.astype(np.float32) / 127.5 - 1 

X_train = np.expand_dims(X_train, axis=3) 

print(X_train.shape) 


4422102/4422102 [==============================] - 2s 0us/step
(60000, 28, 28, 1)


#### Generator

In [4]:
def define_generator(latent_dim):  

    init  = RandomNormal(stddev=0.02) 

    in_lat = Input(shape=(latent_dim,))  

    gen = Dense(256, kernel_initializer=init)(in_lat) 

    gen = LeakyReLU(alpha=0.2)(gen) 

    gen = Dense(512, kernel_initializer=init)(gen) 

    gen = LeakyReLU(alpha=0.2)(gen) 

    gen = Dense(1024, kernel_initializer=init)(gen) 

    gen = LeakyReLU(alpha=0.2)(gen) 

    gen = Dense(28 * 28 * 1, kernel_initializer=init)(gen) 

    out_layer = Activation('tanh')(gen) 

    out_layer = Reshape((28, 28, 1))(gen) 

    model = Model(in_lat, out_layer) 

    return model 

generator = define_generator(100)

#### Discriminator

In [5]:
def define_discriminator(in_shape=(28, 28, 1)): 

    #init = RandomNormal(stddev=0.02)   

    in_image = Input(shape=in_shape) 

    fe = Flatten()(in_image) 

    fe = Dense(1024)(fe) 

    fe = LeakyReLU(alpha=0.2)(fe) 

    fe = Dropout(0.3)(fe) 

    fe = Dense(512)(fe) 

    fe = LeakyReLU(alpha=0.2)(fe) 

    fe = Dropout(0.3)(fe) 

    fe = Dense(256)(fe) 

    fe = LeakyReLU(alpha=0.2)(fe) 

    fe = Dropout(0.3)(fe) 

    out = Dense(1, activation='sigmoid')(fe) 

    model = Model(in_image, out) 

    opt = Adam(lr=0.0002, beta_1=0.5)  

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy']) 

    return model 

discriminator = define_discriminator() 

C:\Users\shah\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


#### GAN architecture

In [6]:
def define_gan(g_model, d_model): 

    d_model.trainable = False 

    gan_output = d_model(g_model.output) 

    model = Model(g_model.input, gan_output) 

    opt = Adam(lr=0.0002, beta_1=0.5) 

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy']) 

    return model 

gan_model = define_gan(generator, discriminator) 

In [7]:
def generate_latent_points(latent_dim, n_samples): 

    x_input = randn(latent_dim * n_samples)   

    z_input = x_input.reshape(n_samples, latent_dim) 

    return z_input 

def generate_real_samples(X_train, n_samples): 

    ix = randint(0, X_train.shape[0], n_samples)  # returns an integer number selected element from the specified range. 

    X = X_train[ix]   

    y = ones((n_samples, 1))  

    return X, y 

def generate_fake_samples(generator, latent_dim, n_samples): 

    z_input = generate_latent_points(latent_dim, n_samples) 

    images = generator.predict(z_input)   

    y = zeros((n_samples, 1)) 

    return images, y 

def summarize_performance(step, g_model, latent_dim, n_samples=100): 

    X, _ = generate_fake_samples(g_model, latent_dim, n_samples) 

    X = (X + 1) / 2.0 

    for i in range(100): 

        pyplot.subplot(10, 10, 1 + i) 

        pyplot.axis('off') 

        pyplot.imshow(X[i, :, :, 0], cmap='gray_r') 

    filename2 = 'model_%04d.h5' % (step+1) 

    g_model.save(filename2) 

    print('>Saved: %s' % (filename2)) 

def save_plot(examples, n_examples): 

    for i in range(n_examples): 

        pyplot.subplot(sqrt(n_examples), sqrt(n_examples), 1 + i) 

        pyplot.axis('off') 

        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r') 

    pyplot.show() 


In [8]:
def train(g_model, d_model, gan_model, X_train, latent_dim, n_epochs=100, n_batch=1000): 

    bat_per_epo = int(X_train.shape[0] / n_batch)

    n_steps = bat_per_epo * n_epochs 

    for i in range(n_steps): 

        X_real, y_real = generate_real_samples(X_train, n_batch) 

        d_loss_r, d_acc_r = d_model.train_on_batch(X_real, y_real) 

        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch) 

        d_loss_f, d_acc_f = d_model.train_on_batch(X_fake, y_fake) 

        z_input = generate_latent_points(latent_dim, n_batch)  

        y_gan = ones((n_batch, 1))  

        g_loss, g_acc = gan_model.train_on_batch(z_input, y_gan) 

        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_loss_r,d_acc_r, d_loss_f,d_acc_f, g_loss,g_acc)) 

        if (i+1) % (bat_per_epo * 1) == 0: 

            summarize_performance(i, g_model, latent_dim) 

In [ ]:
latent_dim = 100 

train(generator, discriminator, gan_model, X_train, latent_dim, n_epochs=20, n_batch=64) 

model = load_model('model_18740.h5') 

2/2 [==============================] - 9s 17ms/step
>1, dr[1.057,0.234], df[0.696,0.188], g[0.691,0.781]
2/2 [==============================] - 0s 7ms/step
>2, dr[0.171,1.000], df[0.706,0.000], g[0.681,1.000]
2/2 [==============================] - 0s 9ms/step
>3, dr[0.045,1.000], df[0.715,0.000], g[0.673,1.000]
2/2 [==============================] - 0s 9ms/step
>4, dr[0.031,1.000], df[0.729,0.000], g[0.663,1.000]
2/2 [==============================] - 0s 9ms/step
>5, dr[0.022,1.000], df[0.744,0.000], g[0.650,1.000]
2/2 [==============================] - 0s 9ms/step
>6, dr[0.018,1.000], df[0.770,0.000], g[0.629,1.000]
2/2 [==============================] - 0s 10ms/step
>7, dr[0.018,1.000], df[0.797,0.000], g[0.611,1.000]
2/2 [==============================] - 0s 8ms/step
>8, dr[0.016,1.000], df[0.833,0.000], g[0.588,1.000]
2/2 [==============================] - 0s 9ms/step
>9, dr[0.017,1.000], df[0.855,0.000], g[0.580,1.000]
2/2 [==============================] - 0s 7ms/step
>10, dr[0.0

2/2 [==============================] - 0s 7ms/step
>79, dr[0.000,1.000], df[0.098,1.000], g[2.831,0.000]
2/2 [==============================] - 0s 8ms/step
>80, dr[0.000,1.000], df[0.096,1.000], g[2.840,0.000]
2/2 [==============================] - 0s 7ms/step
>81, dr[0.004,1.000], df[0.092,1.000], g[2.848,0.000]
2/2 [==============================] - 0s 8ms/step
>82, dr[0.000,1.000], df[0.096,1.000], g[2.935,0.000]
2/2 [==============================] - 0s 10ms/step
>83, dr[0.000,1.000], df[0.081,1.000], g[3.140,0.000]
2/2 [==============================] - 0s 10ms/step
>84, dr[0.007,1.000], df[0.092,1.000], g[3.064,0.000]
2/2 [==============================] - 0s 7ms/step
>85, dr[0.001,1.000], df[0.082,1.000], g[3.366,0.000]
2/2 [==============================] - 0s 8ms/step
>86, dr[0.002,1.000], df[0.074,1.000], g[3.614,0.000]
2/2 [==============================] - 0s 9ms/step
>87, dr[0.001,1.000], df[0.067,1.000], g[4.058,0.000]
2/2 [==============================] - 0s 6ms/step
>8

2/2 [==============================] - 0s 8ms/step
>156, dr[0.002,1.000], df[0.012,1.000], g[5.909,0.000]
2/2 [==============================] - 0s 7ms/step
>157, dr[0.055,0.969], df[0.111,0.984], g[4.156,0.000]
2/2 [==============================] - 0s 8ms/step
>158, dr[0.007,1.000], df[0.180,0.984], g[4.677,0.000]
2/2 [==============================] - 0s 8ms/step
>159, dr[0.259,0.922], df[0.238,0.953], g[4.523,0.000]
2/2 [==============================] - 0s 8ms/step
>160, dr[0.001,1.000], df[0.034,1.000], g[4.756,0.000]
2/2 [==============================] - 0s 9ms/step
>161, dr[0.016,1.000], df[0.096,1.000], g[4.364,0.000]
2/2 [==============================] - 0s 8ms/step
>162, dr[0.007,1.000], df[0.118,1.000], g[4.924,0.000]
2/2 [==============================] - 0s 7ms/step
>163, dr[0.052,0.969], df[0.100,1.000], g[4.538,0.000]
2/2 [==============================] - 0s 8ms/step
>164, dr[0.028,0.984], df[0.180,0.969], g[6.076,0.000]
2/2 [==============================] - 0s 8ms/

2/2 [==============================] - 0s 11ms/step
>233, dr[0.110,0.953], df[0.111,1.000], g[5.493,0.000]
2/2 [==============================] - 0s 7ms/step
>234, dr[0.016,0.984], df[0.031,1.000], g[5.531,0.000]
2/2 [==============================] - 0s 8ms/step
>235, dr[0.086,0.969], df[0.280,0.922], g[8.165,0.000]
2/2 [==============================] - 0s 8ms/step
>236, dr[0.137,0.969], df[0.003,1.000], g[6.724,0.000]
2/2 [==============================] - 0s 10ms/step
>237, dr[0.003,1.000], df[0.021,1.000], g[4.681,0.000]
2/2 [==============================] - 0s 8ms/step
>238, dr[0.006,1.000], df[0.214,0.969], g[6.647,0.000]
2/2 [==============================] - 0s 7ms/step
>239, dr[0.083,0.969], df[0.034,1.000], g[5.287,0.000]
2/2 [==============================] - 0s 7ms/step
>240, dr[0.023,1.000], df[0.165,0.969], g[6.413,0.000]
2/2 [==============================] - 0s 9ms/step
>241, dr[0.232,0.953], df[0.118,1.000], g[5.549,0.000]
2/2 [==============================] - 0s 7m

2/2 [==============================] - 0s 8ms/step
>310, dr[0.009,1.000], df[0.036,1.000], g[5.085,0.000]
2/2 [==============================] - 0s 8ms/step
>311, dr[0.043,0.984], df[0.098,0.984], g[5.873,0.000]
2/2 [==============================] - 0s 9ms/step
>312, dr[0.107,0.953], df[0.060,1.000], g[5.325,0.000]
2/2 [==============================] - 0s 8ms/step
>313, dr[0.352,0.938], df[0.936,0.469], g[10.455,0.000]
2/2 [==============================] - 0s 9ms/step
>314, dr[0.968,0.812], df[0.002,1.000], g[5.746,0.000]
2/2 [==============================] - 0s 7ms/step
>315, dr[0.001,1.000], df[0.027,1.000], g[3.968,0.000]
2/2 [==============================] - 0s 8ms/step
>316, dr[0.004,1.000], df[0.126,1.000], g[4.074,0.000]
2/2 [==============================] - 0s 7ms/step
>317, dr[0.007,1.000], df[0.049,1.000], g[4.089,0.000]
2/2 [==============================] - 0s 8ms/step
>318, dr[0.008,1.000], df[0.084,0.984], g[4.397,0.000]
2/2 [==============================] - 0s 10m

...

2/2 [==============================] - 1s 244ms/step
>16600, dr[0.637,0.609], df[0.538,0.828], g[0.900,0.250]
2/2 [==============================] - 1s 250ms/step
>16601, dr[0.592,0.594], df[0.705,0.656], g[0.985,0.203]
2/2 [==============================] - 1s 255ms/step
>16602, dr[0.645,0.562], df[0.690,0.625], g[0.875,0.250]
2/2 [==============================] - 1s 261ms/step
>16603, dr[0.690,0.484], df[0.622,0.750], g[1.003,0.141]
2/2 [==============================] - 1s 259ms/step
>16604, dr[0.710,0.469], df[0.615,0.734], g[0.953,0.125]
2/2 [==============================] - 1s 247ms/step
>16605, dr[0.679,0.547], df[0.517,0.828], g[0.914,0.281]
2/2 [==============================] - 1s 265ms/step
>16606, dr[0.734,0.422], df[0.648,0.625], g[1.012,0.156]
2/2 [==============================] - 0s 217ms/step
>16607, dr[0.665,0.531], df[0.628,0.703], g[0.961,0.188]
2/2 [==============================] - 1s 238ms/step
>16608, dr[0.594,0.641], df[0.584,0.781], g[0.981,0.203]
2/2 [=====

>16674, dr[0.587,0.656], df[0.564,0.750], g[1.016,0.234]
2/2 [==============================] - 1s 225ms/step
>16675, dr[0.560,0.688], df[0.606,0.672], g[0.958,0.250]
2/2 [==============================] - 1s 250ms/step
>16676, dr[0.713,0.469], df[0.582,0.734], g[1.064,0.172]
2/2 [==============================] - 1s 252ms/step
>16677, dr[0.688,0.547], df[0.591,0.719], g[1.025,0.219]
2/2 [==============================] - 1s 250ms/step
>16678, dr[0.622,0.656], df[0.622,0.703], g[0.990,0.234]
2/2 [==============================] - 1s 223ms/step
>16679, dr[0.674,0.516], df[0.624,0.719], g[0.981,0.219]
2/2 [==============================] - 1s 241ms/step
>16680, dr[0.629,0.594], df[0.614,0.719], g[1.054,0.156]
2/2 [==============================] - 1s 249ms/step
>16681, dr[0.630,0.547], df[0.574,0.734], g[0.981,0.203]
2/2 [==============================] - 1s 234ms/step
>16682, dr[0.616,0.609], df[0.714,0.594], g[0.959,0.250]
2/2 [==============================] - 1s 250ms/step
>16683, dr

2/2 [==============================] - 1s 257ms/step
>16749, dr[0.528,0.703], df[0.623,0.703], g[0.961,0.172]
2/2 [==============================] - 1s 239ms/step
>16750, dr[0.581,0.688], df[0.625,0.719], g[1.001,0.141]
2/2 [==============================] - 1s 252ms/step
>16751, dr[0.643,0.547], df[0.564,0.703], g[1.030,0.156]
2/2 [==============================] - 1s 252ms/step
>16752, dr[0.732,0.500], df[0.577,0.766], g[1.021,0.156]
2/2 [==============================] - 1s 240ms/step
>16753, dr[0.668,0.469], df[0.628,0.719], g[1.000,0.188]
2/2 [==============================] - 1s 254ms/step
>16754, dr[0.566,0.734], df[0.579,0.750], g[1.020,0.141]
2/2 [==============================] - 1s 259ms/step
>16755, dr[0.692,0.469], df[0.723,0.656], g[0.983,0.172]
2/2 [==============================] - 1s 262ms/step
>16756, dr[0.665,0.547], df[0.640,0.656], g[0.973,0.172]
2/2 [==============================] - 1s 256ms/step
>16757, dr[0.638,0.516], df[0.576,0.797], g[1.036,0.109]
2/2 [=====

>16823, dr[0.662,0.500], df[0.674,0.625], g[0.958,0.250]
2/2 [==============================] - 1s 235ms/step
>16824, dr[0.627,0.531], df[0.564,0.734], g[0.907,0.234]
2/2 [==============================] - 1s 248ms/step
>16825, dr[0.615,0.625], df[0.604,0.703], g[0.977,0.125]
2/2 [==============================] - 1s 260ms/step
>16826, dr[0.623,0.609], df[0.603,0.734], g[0.943,0.266]
2/2 [==============================] - 1s 269ms/step
>16827, dr[0.686,0.578], df[0.720,0.609], g[0.958,0.203]
2/2 [==============================] - 1s 241ms/step
>16828, dr[0.658,0.500], df[0.614,0.703], g[1.039,0.094]
2/2 [==============================] - 1s 255ms/step
>16829, dr[0.671,0.531], df[0.573,0.750], g[1.033,0.094]
2/2 [==============================] - 1s 256ms/step
>16830, dr[0.715,0.469], df[0.566,0.766], g[1.037,0.125]
2/2 [==============================] - 1s 253ms/step
>16831, dr[0.553,0.641], df[0.617,0.703], g[0.943,0.234]
2/2 [==============================] - 1s 281ms/step
>16832, dr

2/2 [==============================] - 0s 220ms/step
>16896, dr[0.648,0.562], df[0.625,0.766], g[0.953,0.250]
2/2 [==============================] - 0s 215ms/step
>16897, dr[0.645,0.594], df[0.898,0.547], g[1.004,0.234]
2/2 [==============================] - 1s 232ms/step
>16898, dr[0.605,0.594], df[0.658,0.719], g[1.140,0.062]
2/2 [==============================] - 1s 248ms/step
>16899, dr[0.643,0.609], df[0.621,0.719], g[1.033,0.188]
2/2 [==============================] - 1s 254ms/step
>16900, dr[0.678,0.531], df[0.571,0.797], g[1.001,0.156]
2/2 [==============================] - 1s 245ms/step
>16901, dr[0.680,0.500], df[0.634,0.688], g[0.950,0.188]
2/2 [==============================] - 1s 239ms/step
>16902, dr[0.692,0.516], df[0.626,0.766], g[1.039,0.125]
2/2 [==============================] - 0s 220ms/step
>16903, dr[0.695,0.531], df[0.537,0.828], g[0.975,0.141]
2/2 [==============================] - 1s 251ms/step
>16904, dr[0.634,0.516], df[0.650,0.703], g[1.076,0.047]
2/2 [=====

In [ ]:
latent_dim = 100 
n_examples = 100 
latent_points = generate_latent_points(latent_dim, n_examples) 
X  = model.predict(latent_points) 
X = (X + 1) / 2.0
save_plot(X, n_examples) 